In [3]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

In [4]:
from utils import generate_dataset, ImplicitDataset, TestDataset, SliceDataset
import numpy as np
from sdf import *
import math

@sdf3
def gyroid(w = 3.14159, t=0):
    def f(p):
        q = w*p
        x, y, z = (q[:, i] for i in range(3))
        return (np.cos(x)*np.sin(y) + np.cos(y)*np.sin(z) + np.cos(z)*np.sin(x) - t)
    return f

In [10]:
generate_dataset(box(1.0) & gyroid(w=math.pi*4, t=0), N_train=2e4, N_test=1e5, name='box_1f0_gyroid_4pi', save_dir='../datasets')

Saved file at ../datasets\box_1f0_gyroid_4pi.stl
ImplicitDataset (20000 PDE points, 6025 Dirichlet BCs)
SliceDataset (10000 points)
UniformMeshSDFDataset (97336 points)
RandomMeshSDFDataset (100000 points)


In [11]:
name = 'box_1f0_gyroid_4pi'
train = ImplicitDataset.from_file(f'../datasets/{name}_train.npz')
test = TestDataset(f'../datasets/{name}_test.npz')
slice_dataset = SliceDataset.from_file(f'../datasets/{name}_slice.npz')

In [12]:
print(train)
print(test)
print(slice_dataset)

ImplicitDataset (20000 PDE points, 6025 Dirichlet BCs)
TestDataset (97336 points, 100000 points)
SliceDataset (10000 points)


In [13]:
from torch.utils.data import DataLoader
train_residual_dataloader = DataLoader(train.residual, batch_size=4096, shuffle=True)
train_bc_dataloader = DataLoader(train.bc, batch_size=4096, shuffle=True)

In [14]:
for _batch in train_bc_dataloader:
    print(_batch['x'].shape, _batch['bc'].shape)

torch.Size([4096, 3]) torch.Size([4096])
torch.Size([1929, 3]) torch.Size([1929])
